In [25]:
import pandas as pd 
import numpy as np

In [26]:
df=pd.read_csv('bank-full.csv', sep=';')

In [27]:
df

,age,job,marital,education,default,balance,housing,loan,contact,day,month,duration,campaign,pdays,previous,poutcome,y
0,58,management,married,tertiary,no,2143,yes,no,unknown,5,may,261,1,-1,0,unknown,no
1,44,technician,single,secondary,no,29,yes,no,unknown,5,may,151,1,-1,0,unknown,no
2,33,entrepreneur,married,secondary,no,2,yes,yes,unknown,5,may,76,1,-1,0,unknown,no
3,47,blue-collar,married,unknown,no,1506,yes,no,unknown,5,may,92,1,-1,0,unknown,no
4,33,unknown,single,unknown,no,1,no,no,unknown,5,may,198,1,-1,0,unknown,no
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
45206,51,technician,married,tertiary,no,825,no,no,cellular,17,nov,977,3,-1,0,unknown,yes
45207,71,retired,divorced,primary,no,1729,no,no,cellular,17,nov,456,2,-1,0,unknown,yes
45208,72,retired,married,secondary,no,5715,no,no,cellular,17,nov,1127,5,184,3,success,yes
45209,57,blue-collar,married,secondary,no,668,no,no,telephone,17,nov,508,4,-1,0,unknown,no


In [28]:
data=df[['age',
'job',
'marital',
'education',
'balance',
'housing',
'contact',
'day',
'month',
'duration',
'campaign',
'pdays',
'previous',
'poutcome',
'y']]

In [29]:
data

,age,job,marital,education,balance,housing,contact,day,month,duration,campaign,pdays,previous,poutcome,y
0,58,management,married,tertiary,2143,yes,unknown,5,may,261,1,-1,0,unknown,no
1,44,technician,single,secondary,29,yes,unknown,5,may,151,1,-1,0,unknown,no
2,33,entrepreneur,married,secondary,2,yes,unknown,5,may,76,1,-1,0,unknown,no
3,47,blue-collar,married,unknown,1506,yes,unknown,5,may,92,1,-1,0,unknown,no
4,33,unknown,single,unknown,1,no,unknown,5,may,198,1,-1,0,unknown,no
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
45206,51,technician,married,tertiary,825,no,cellular,17,nov,977,3,-1,0,unknown,yes
45207,71,retired,divorced,primary,1729,no,cellular,17,nov,456,2,-1,0,unknown,yes
45208,72,retired,married,secondary,5715,no,cellular,17,nov,1127,5,184,3,success,yes
45209,57,blue-collar,married,secondary,668,no,telephone,17,nov,508,4,-1,0,unknown,no


In [30]:
from sklearn.model_selection import train_test_split
data_full_train,data_test=train_test_split(data,test_size=0.2,random_state=1)
data_train,data_val=train_test_split(data,test_size=0.25,random_state=1)

In [31]:
numerical=['balance',
'day',
'duration',
'previous']

In [32]:
categorical=['age',
'job',
'marital',
'education',
'housing',
'contact',
'month',
'campaign',
'pdays',
'poutcome',
'y']

In [33]:
data_train['y'] = data_train['y'].map({'yes': 1, 'no': 0})
data_val['y'] = data_val['y'].map({'yes': 1, 'no': 0})
data_test['y'] = data_test['y'].map({'yes': 1, 'no': 0})

In [34]:
y_train=data_train['y'].values
y_val=data_val['y'].values
y_test=data_test['y'].values

In [38]:
from sklearn.metrics import roc_auc_score
ROC_AUC_feature_importance=[]
for num in numerical:
    score=roc_auc_score(y_train,data_train[num])
    ROC_AUC_feature_importance.append((num, score))

In [39]:
ROC_AUC_feature_importance

[('balance', np.float64(0.5925039928796861)),
 ('day', np.float64(0.47286318110297093)),
 ('duration', np.float64(0.8117171432689722)),
 ('previous', np.float64(0.5994010001038775))]